# Bootstrap

## 準備

In [1]:
import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot
import warnings


# warningを非表示
warnings.filterwarnings('ignore')

# numpyは小数第3位まで表示
np.set_printoptions(precision=3, suppress=True)

# 乱数を設定
np.random.seed(0)

## テストデータ

In [2]:
# 各変数ごとにデータ生成
x3 = np.random.uniform(size=1000)
x0 = 3.0*x3 + np.random.uniform(size=1000)
x2 = 6.0*x3 + np.random.uniform(size=1000)
x1 = 3.0*x0 + 2.0*x2 + np.random.uniform(size=1000)
x5 = 4.0*x0 + np.random.uniform(size=1000)
x4 = 8.0*x0 - 1.0*x2 + np.random.uniform(size=1000)

# DataFrameとして格納
X = pd.DataFrame(np.array([x0, x1, x2, x3, x4, x5]).T ,columns=['x0', 'x1', 'x2', 'x3', 'x4', 'x5'])

## ブートストラップ法

In [3]:
# DirectLiNGAMのオブジェクト
model = lingam.DirectLiNGAM()

# ブートストラップ法（サンプリング数100）で学習
result = model.bootstrap(X, n_sampling=100)

## 因果方向

In [4]:
# ブートストラップ法の結果に基づき因果の矢印を確度の高い順で集計
# n_directions: 集計する因果の矢印の数（ランキング順）
# min_causal_effect: 集計する係数（因果効果）の最小値
# split_by_causal_effect_sign: 係数（因果効果）の符号を区別するかどうか
cdc = result.get_causal_direction_counts(n_directions=8, min_causal_effect=0.01, split_by_causal_effect_sign=True)

In [5]:
# 集計結果の表示
# 第2引数（=100）はブートストラップ法のサンプリング数を入力
print_causal_directions(cdc, 100)

x5 <--- x0 (b>0) (100.0%)
x1 <--- x0 (b>0) (100.0%)
x1 <--- x2 (b>0) (100.0%)
x4 <--- x2 (b<0) (100.0%)
x0 <--- x3 (b>0) (99.0%)
x4 <--- x0 (b>0) (98.0%)
x2 <--- x3 (b>0) (96.0%)
x1 <--- x5 (b>0) (56.0%)


## 有向非巡回グラフ

In [6]:
# ブートストラップ法の結果に基づき因果構造を確度の高い順で集計
# n_dags: 集計する因果構造の数（ランキング順）
# min_causal_effect: 考慮する係数（因果効果）の最小値
# split_by_causal_effect_sign: 係数（因果効果）の符号を区別するかどうか
dagc = result.get_directed_acyclic_graph_counts(n_dags=3, min_causal_effect=0.01, split_by_causal_effect_sign=True)

In [7]:
# 集計結果の表示
# 第2引数（=100）はブートストラップサンプル数を入力
print_dagc(dagc, 100)

DAG[0]: 6.0%
	x0 <--- x3 (b>0)
	x1 <--- x0 (b>0)
	x1 <--- x2 (b>0)
	x2 <--- x3 (b>0)
	x4 <--- x0 (b>0)
	x4 <--- x2 (b<0)
	x5 <--- x0 (b>0)
	x5 <--- x3 (b>0)
DAG[1]: 5.0%
	x0 <--- x3 (b>0)
	x1 <--- x0 (b>0)
	x1 <--- x2 (b>0)
	x2 <--- x3 (b>0)
	x4 <--- x0 (b>0)
	x4 <--- x2 (b<0)
	x5 <--- x0 (b>0)
DAG[2]: 4.0%
	x0 <--- x3 (b>0)
	x1 <--- x0 (b>0)
	x1 <--- x2 (b>0)
	x1 <--- x5 (b>0)
	x2 <--- x0 (b>0)
	x2 <--- x3 (b>0)
	x4 <--- x0 (b>0)
	x4 <--- x2 (b<0)
	x5 <--- x0 (b>0)
	x5 <--- x3 (b>0)


## 出現確率

In [8]:
# ブートストラップ法の結果に基づき各因果の矢印の出現確率を集計
# min_causal_effect: 考慮する係数（因果効果）の最小値
prob = result.get_probabilities(min_causal_effect=0.01)

# 集計結果の表示
print(prob)

[[0.   0.   0.1  0.99 0.02 0.  ]
 [1.   0.   1.   0.11 0.07 0.56]
 [0.4  0.   0.   0.96 0.   0.04]
 [0.   0.   0.04 0.   0.   0.  ]
 [0.98 0.01 1.   0.11 0.   0.36]
 [1.   0.01 0.11 0.56 0.15 0.  ]]


## （総合）因果効果

In [9]:
# ブートストラップ法の結果に基づき（総合）因果効果を計算
# min_causal_effect: 考慮する係数（因果効果）の最小値
causal_effects = result.get_total_causal_effects(min_causal_effect=0.01)

# DataFrameに結果をまとめる
df = pd.DataFrame(causal_effects)
labels = [f'x{i}' for i in range(X.shape[1])]
df['from'] = df['from'].apply(lambda x : labels[x])
df['to'] = df['to'].apply(lambda x : labels[x])

In [10]:
# （総合）因果効果の大きい順にトップ5を表示
print(df.sort_values('effect', ascending=False).head())

  from  to     effect  probability
4   x3  x1  20.931938         1.00
5   x3  x4  18.077244         1.00
6   x3  x5  12.024250         1.00
8   x0  x4   7.993145         0.98
9   x3  x2   5.970163         0.96


In [11]:
# 存在確率の小さい順にトップ5を表示
print(df.sort_values('probability', ascending=True).head())

   from  to    effect  probability
18   x4  x5  0.002161         0.02
16   x4  x1  0.365265         0.02
17   x4  x0  0.123074         0.02
15   x2  x3  0.163050         0.04
14   x2  x5  1.961195         0.04


In [12]:
# x1に向かう（総合）因果効果を表示
print(df[df['to']=='x1'].head())

   from  to     effect  probability
2    x0  x1   3.004868         1.00
3    x2  x1   2.092102         1.00
4    x3  x1  20.931938         1.00
13   x5  x1   0.021796         0.14
16   x4  x1   0.365265         0.02


## 経路の存在確率と（総合）因果効果

In [13]:
# ブートストラップ法の結果に基づき経路の存在確率と（総合）因果効果を計算
from_index = 3 # 原因となる変数のインデックス（x3）
to_index = 1 # 結果となる変数のインデックス（x1）

# 存在確率の大きい順にトップ5を表示
print(pd.DataFrame(result.get_paths(from_index, to_index)).head())

           path     effect  probability
0     [3, 0, 1]   8.567958         0.99
1     [3, 2, 1]  11.943522         0.96
2  [3, 0, 5, 1]   0.413070         0.63
3  [3, 0, 2, 1]   0.249261         0.40
4     [3, 5, 1]   0.005053         0.36
